In [11]:
import wave
from pyaudio import PyAudio,paInt16
import os
from aip import AipSpeech
import ffmpy #视频音频转码模块
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re


os.chdir('E:\我的东西\新建文件夹\文件\课件\信工课件\电子系统综合设计\半成品\代码文件')
framerate=8000
NUM_SAMPLES=2000
channels=1
sampwidth=2
TIME=1
filename_record = 'audio_record.wav'
filename_answer = 'audio.wav'
APP_ID = '11169887'
API_KEY = 'TQypLIsDnr4XwzfyKGLqMsfD'
SECRET_KEY = 'bc5efee36b796c2b467dba12c2a080b0'


def save_wave_file(filename,data):
    '''save the date to the wavfile'''
    wf=wave.open(filename,'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(sampwidth)
    wf.setframerate(framerate)
    wf.writeframes(b"".join(data))
    wf.close()

def my_record():
    pa=PyAudio()
    stream=pa.open(format = paInt16,channels=1,
                   rate=framerate,input=True,
                   frames_per_buffer=NUM_SAMPLES)
    my_buf=[]
    count=0
    while count<TIME*20:#控制录音时间
        string_audio_data = stream.read(NUM_SAMPLES)
        my_buf.append(string_audio_data)
        count+=1
        print('.',end='')
    stream.close()
    return my_buf

chunk=2048 #好像是分段记录时每次记录的点数
def play(filename):
    wf=wave.open(filename,'rb')  
    p=PyAudio()
    stream=p.open(format=p.get_format_from_width(wf.getsampwidth()),channels=
    wf.getnchannels(),rate=wf.getframerate(),output=True)
    while True:
        data=wf.readframes(chunk)
        if data==b'':break
        stream.write(data) #这一行是播放语音
    stream.close()
    p.terminate()

# 读取文件
def get_file_content(filePath):
    with open(filePath, 'rb') as fp:
        return fp.read() 

def get_weather():
    resp=urlopen('http://www.weather.com.cn/weather/101280101.shtml')
    soup=BeautifulSoup(resp,'html.parser')
    tagDate=soup.find('ul', class_="t clearfix")
    dates=tagDate.h1.string
    tagToday=soup.find('p', class_="tem")
    try:
        temperatureHigh=tagToday.span.string
    except AttributeError as e:
        temperatureHigh=tagToday.find_next('p', class_="tem").span.string
    temperatureLow=tagToday.i.string
    weather=soup.find('p', class_="wea").string
    tagWind=soup.find('p',class_="win")
    winL=tagWind.i.string
    result_weather = '今天是：'+ dates.replace('（今天）','') + ',风级：' + winL + ',最低温度：' + temperatureLow.replace('℃','度') + ',最高温度：' + temperatureHigh.replace('℃','度') + ',天气：'+ weather
    return result_weather

client = AipSpeech(APP_ID, API_KEY, SECRET_KEY) 
filename_record = 'audio_record.wav'
data = my_record()
save_wave_file(filename_record,data)

# message = result_weather.encode('utf8')
# wf=wave.open(filename_record,'rb')
# rate=wf.getframerate()    #照目前的情况是8000，如果有变化，再用上这个语句
result_stt = client.asr(get_file_content(filename_record), 'wav', 8000, {'dev_pid': '1537'}) #dev_pid=1537是纯中文普通话，1536支持简单英文
# wf.close()
print(result_stt['result'])
message = str(result_stt['result']).encode('utf8')
search_result = re.search(u'.*魔镜魔镜'.encode('utf8'), message)
if search_result is not None: #接收到了唤醒语，开始工作
    flag = 1
    search_result = re.search(u'.*天气'.encode('utf8'), message)
    if search_result is not None:
        message_answer = get_weather()
    else:
        flag = 0
#         message_answer = '你在说什么我听不见'
    if flag:
        result_tts = client.synthesis(message_answer, 'zh', 1, {'vol': 5, 'per': 3}) 
        if not isinstance(result_tts, dict):
            with open('audio_raw.wav', 'wb') as f:
                f.write(result_tts)
        ff = ffmpy.FFmpeg(
            inputs={'audio_raw.wav': None},
            outputs={filename_answer: None}
        )
        ff.run() #美中不足的时没有学会怎么执行覆盖文件操作，文件存在时写入操作会被直接跳过，故现在只能暂时在转码播放后直接将其移除
        play(filename_answer)
        os.remove(filename_answer)

# 首先实现：读取A23_0.wav，能将识别内容显示在屏幕上（已完成）
# 然后实现：将显示在屏幕上的识别内容转成音频
# 接着给音频加上RIFF头
# 最后播放音频

....................['我就，']


In [10]:
result_stt

{'corpus_no': '6550914675136817954',
 'err_msg': 'success.',
 'err_no': 0,
 'result': ['魔镜魔镜今天天气怎么样，'],
 'sn': '200189269881525253680'}

In [8]:
import sys

In [4]:
sys.path

['',
 'D:\\Anaconda3 4.2.0\\python35.zip',
 'D:\\Anaconda3 4.2.0\\DLLs',
 'D:\\Anaconda3 4.2.0\\lib',
 'D:\\Anaconda3 4.2.0',
 'D:\\Anaconda3 4.2.0\\lib\\site-packages',
 'D:\\Anaconda3 4.2.0\\lib\\site-packages\\Sphinx-1.4.6-py3.5.egg',
 'D:\\Anaconda3 4.2.0\\lib\\site-packages\\win32',
 'D:\\Anaconda3 4.2.0\\lib\\site-packages\\win32\\lib',
 'D:\\Anaconda3 4.2.0\\lib\\site-packages\\Pythonwin',
 'D:\\Anaconda3 4.2.0\\lib\\site-packages\\setuptools-27.2.0-py3.5.egg',
 'D:\\Anaconda3 4.2.0\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\Administrator\\.ipython']

In [12]:
import pyaudio
import pprint
import numpy as np
from scipy.io.wavfile import read
a = read('audio_record.wav')
b = np.array(a[1], dtype=float)
#b 就是数据矩阵

40000